In [12]:
import pandas as pd
import warnings
import os
from pandasgui import show
warnings.filterwarnings('ignore')

In [13]:
#Se cargaran los directorios de los csv de precios de los distintos tipos de propieades y los alquileres, separandolos en dos listas distintas.
#Los cuales se encuentran en las carpetas "Mediana_Alquiler_Estados" y "Property_Precio_Estados"
carpeta_alquiler = "Mediana_Alquiler_Estados"
carpeta_precio = "Property_Precio_Estados"
lista_dataset_alquiler = os.listdir(carpeta_alquiler)
lista_dataset_precio = os.listdir(carpeta_precio)


    

In [14]:
#Creamos un dataframe vacio donde iran siendo añadidos uno a uno los demas dataset

final_dataset = pd.DataFrame(columns=["region","date","price","rent","property_type"])

Creamos un loop For en el cual se iteraran simultaneamente ambas listas de directorios, para asi obtener de a pares, los precios del alquiler y
precios de la propiedad y unificarlo todo en un mismo dataset, seperado por tipo de propiedad.

In [15]:
#Cargamos los datasets de alquiler y precio a un dataframe cada uno
for rent,price in zip(lista_dataset_alquiler,lista_dataset_precio):
    #Obtenemos el tipo de propiedad
    property_type = rent[24:(rent.find("."))]
    #utiizamos los directorios
    rent = pd.read_csv(carpeta_alquiler+"/"+rent)
    price = pd.read_csv(carpeta_precio+"/"+price)
    #creamos unos dataframes vacios para ir cargando los datasets de alquiler y precio que estemos recorriendo en el momento ya limpios
    df = pd.DataFrame(columns=["region","date","rent"])
    df1 = pd.DataFrame(columns=["region","date","price"])
    #Eliminamos columnas innecesarias
    rent = rent.drop(["Unnamed: 0","SizeRank"],axis=1)
    price = price.drop(["Unnamed: 0","RegionID","SizeRank","RegionType","StateName"],axis=1)
    price = price.drop(price.iloc[:,[-1,-2,-3]],axis=1)
    price=price.drop(price.columns[1:170],axis=1)
    #Cambiamos el formato de el dataframe rent 
    for element in rent["RegionName"]:
     skip = False
     for element1 in rent.columns:
         if skip == False:
             pass
             skip = True
         else:
             rent_price = float(rent[rent["RegionName"] == element][element1])
             lista = [element,element1,rent_price]
             df = df.append(pd.DataFrame([lista], columns=df.columns), ignore_index=True)
    #Hacemos lo mismo con el dataframe de price
    for element in price["RegionName"]:
    
     skip = False
     for element1 in price.columns:
         if skip == False:
             pass
             skip = True
         else:
             property_price = float(price[price["RegionName"] == element][element1])
             lista = [element,element1,property_price]
             df1 = df1.append(pd.DataFrame([lista], columns=df1.columns), ignore_index=True)
    #Cambiamos el formato de la fecha del Dataframe de price, para que coincida con el de rent
    df1["date"]=(df1["date"]).astype(str)
    df1["date"] = df1["date"].apply(lambda x: x[0:7])

    #Combinamos el dataframe de rent con el de price
    df2 = df1.merge(df)
    #Le añadimos el tipo de propiedad
    df2["property_type"] = property_type

    #Agregamos este dataframe al dataframe final
    final_dataset = final_dataset.append(df2,ignore_index=True)


In [16]:
#Remplazamos los valores faltantes con 0
final_dataset = final_dataset.fillna(0)

In [17]:
#Cambiamos la columna date a formato fecha y creamos una columnna de año
final_dataset["date"] = pd.to_datetime(final_dataset["date"],format="%Y-%m")
final_dataset["year"] = pd.DatetimeIndex(final_dataset["date"]).year
final_dataset["year"] = final_dataset["year"].astype(int)

In [18]:
#Creamos una funcion la cual rellena el valor de la renta con el promedio de la renta en la mismo valor de region, año y tipo de propiedad
def completar_rent(x):
    if x["rent"] < 1 :
        año = x["year"]
        mean = 0.0
        while mean < 1 :
            mean = final_dataset[(final_dataset["region"]==x["region"]) & (final_dataset["rent"] > 1) & (final_dataset["year"] == año)&(final_dataset["property_type"] == x["property_type"])]["rent"].mean()
            if mean < 1:
                año += 1
            else:
                break
        return(mean)
    else:
        return(x["rent"])

In [32]:
def completar_rent2(x):
    condicion = False
    while condicion == False:
        mean = final_dataset[(final_dataset["region"]==x["region"]) & (final_dataset["rent"] > 1) & (final_dataset["year"] == x["year"])&(final_dataset["property_type"] == x["property_type"])]["rent"].mean()
        

TypeError: 'NAType' object is not callable

In [39]:
final_dataset = final_dataset.fillna(0)

In [40]:
final_dataset[[final_dataset["region"]=="New Mexico"] & [final_dataset["rent"] == 0.0]& [final_dataset["year"] ==2011] & [final_dataset["property_type"]=="1Bedroom"]]


TypeError: unsupported operand type(s) for &: 'list' and 'list'

In [24]:
#aplicamos la funcion sobre la renta
final_dataset["rent"] = final_dataset.apply(completar_rent,axis=1)
final_dataset = final_dataset.fillna(0)
final_dataset["rent"] = final_dataset.apply(completar_rent,axis=1)

In [20]:
#calculamos el roi 
def calculate_Roi(x):
    return(x["price"]/x["rent"]/12)

final_dataset["roi"] = final_dataset.apply(calculate_Roi,axis=1)



In [25]:
#previsualizacion de la tabla
show(final_dataset)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [22]:
#Pasamos el dataframe a csv
final_dataset.to_csv("Price_Rent_Roi.csv",index=False)